# Data Preprocessing

In [2]:
#pip install opencv-python
import os
from os import listdir
from os.path import isfile
from os.path import join
from pathlib import Path
import cv2
from math import ceil
import argparse
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import pandas as pd
import yaml


import random
import numpy as np
from torch.utils.tensorboard import SummaryWriter
from utils.func import *
from train import train, evaluate
from utils.metrics import Estimator
from data.builder import generate_dataset
from modules.builder import generate_model
import hydra
from hydra import initialize, compose
from omegaconf import DictConfig



In [3]:
#Input 
dir= "C:/Users/x4287225/Desktop/confounder-free-cnn/OCTDL"
dataset_folder = dir + "/OCTDL images/"
#lables for each Input
labels_path = dir + "/OCTDL_labels.csv"
#Output
output_folder = dir + "/output"
#model save path
save_path = dir + "/run"
#Config file
cg = dir + "/configs/OCTDL.yaml"
config_path = dir + '/configs'
#Defines the ratio for central cropping of images; 1 means no cropping. default=1
crop_ratio = 1
#Specifies the dimensions to which images should be resized Default is 512
image_dim =512
#validation set
val_ratio = 0.15
#test_ratio
test_ratio = 0.25
#A boolean flag that, when true, indicates images should be padded to become square. Default is False
padding = True
#A boolean flag that, when true, indicates images should be cropped centrally according to the crop_ratio Default is False.
crop = False
#A boolean flag that, when true, indicates images should be resized to image_dim. Default is False.
resize =True
labels = ['AMD', 'DME', 'ERM', 'NO',]
folders = ['train', 'val', 'test']

In [4]:

def main():
    root_folder = dataset_folder
    train_ratio = 1 - val_ratio - test_ratio
    dim = (image_dim, image_dim)
    padding_bool = padding
    crop_bool = crop
    resize_bool = resize
    #Creating the output folder
    for folder in folders:
        for label in labels:
            Path(os.path.join(output_folder, folder, label)).mkdir(parents=True, exist_ok=True)
    
    #Setting up Train Test Validation 
    for label in tqdm(labels):
        df_label = df[df['disease'] == label][['file_name', 'disease', 'patient_id']]
        patients_list = df_label.patient_id.unique()
        if len(patients_list) < 10:
            print(f"Skipping {label} due to insufficient data.")
            continue
        train_patients, test_patients = train_test_split(patients_list, test_size=1 - train_ratio)
        val_patients, test_patients = train_test_split(test_patients, test_size=test_ratio / (test_ratio + val_ratio))
        df_label_train = df_label[df_label['patient_id'].isin(train_patients)]
        df_label_val = df_label[df_label['patient_id'].isin(val_patients)]
        df_label_test = df_label[df_label['patient_id'].isin(test_patients)]
        
        print(label, len(df_label_train), len(df_label_val), len(df_label_test))
        #Data Augmentation on the Input data (Resizing, Cropping and Padding)
        for i in range(0, len(df_label_train)):
            file_name = df_label_train.iloc[i, 0] + '.jpg'
            file_label = df_label_train.iloc[i, 1]    
            preprocessing(root_folder, output_folder, file_name, 'train', crop_ratio, dim, file_label, padding_bool, crop_bool, resize_bool)
 
        for i in range(0, len(df_label_test)):
            file_name = df_label_test.iloc[i, 0] + '.jpg'
            file_label = df_label_test.iloc[i, 1]    
            preprocessing(root_folder, output_folder, file_name, 'test', crop_ratio, dim, file_label, padding_bool, crop_bool, resize_bool)    
    
        for i in range(0, len(df_label_val)):
            file_name = df_label_val.iloc[i, 0] + '.jpg'
            file_label = df_label_val.iloc[i, 1]    
            preprocessing(root_folder, output_folder, file_name, 'val', crop_ratio, dim, file_label, padding_bool, crop_bool, resize_bool) 


def preprocessing(root_folder, output_folder, file, folder, crop_ratio, dim, label, padding_bool, crop_bool, resize_bool):
    img = cv2.imread(os.path.join(root_folder, label, file))
    if padding_bool:
        img = padding(img)
    if crop_bool:
        img = center_crop(img, (img.shape[1] * crop_ratio, img.shape[0] * crop_ratio))
    if resize_bool:
        img = cv2.resize(img, dim, interpolation=cv2.INTER_LANCZOS4)    #cv2.INTER_LANCZOS4 is used for reducing or increasing image size while maintaining high image quality.
    cv2.imwrite(os.path.join(output_folder, folder, label, Path(file).name), img, [cv2.IMWRITE_JPEG_QUALITY, 100])  #image is saved with high JPEG quality (100), which means minimal compression and maximal quality.

In [5]:
def padding(img):
    """Returns padded to square image
    """
    height = img.shape[0]
    width = img.shape[1]
    if width == height:
        return img
    elif width > height:
        left = 0
        right = 0
        bottom = ceil((width - height) / 2)
        top = ceil((width - height) / 2)
        result = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=(0, 0, 0))
        return result
    else:
        left = ceil((height - width) / 2)
        right = ceil((height - width) / 2)
        bottom = 0
        top = 0
        result = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=(0, 0, 0))
        return result


def center_crop(img, dim):
    """Returns center cropped image
    """
    width, height = img.shape[1], img.shape[0]
    # process crop width and height for max available dimension
    crop_width = dim[0] if dim[0] < img.shape[1] else img.shape[1]
    crop_height = dim[1] if dim[1] < img.shape[0] else img.shape[0]
    mid_x, mid_y = int(width/2), int(height/2)
    cw2, ch2 = int(crop_width/2), int(crop_height/2)
    crop_img = img[mid_y-ch2:mid_y+ch2, mid_x-cw2:mid_x+cw2]
    return crop_img

def scale_image(img, factor=1):
    """Returns resize image by scale factor.
    This helps to retain resolution ratio while resizing.
    Args:
    img: image to be scaled
    factor: scale factor to resize"
    """
    width = int(img.shape[1] * factor)
    height = int(img.shape[0] * factor)
    dsize = (width, height)
    output = cv2.resize(img, dsize, interpolation=cv2.INTER_LANCZOS4)
    return output

In [6]:
#Reading the Labels
df = pd.read_csv(labels_path)

df= df[df['sex'] != '0']

image_ids = set(df['file_name'])
base_directory = dir +  '/OCTDL images/'
diseases = df['disease'].unique()

for disease in diseases:
    disease_folder_path = os.path.join(dataset_folder, disease)
    
    if not os.path.exists(disease_folder_path):
        print(f"Folder not found: {disease_folder_path}")
        continue
    
    files = os.listdir(disease_folder_path)
    
    for file in files:
        if file.split('.')[0] not in image_ids:
            file_path = os.path.join(disease_folder_path, file)
            os.remove(file_path)
            print(f"Deleted {file_path}")
df = df.groupby('disease').filter(lambda x: x['patient_id'].nunique() >= 10)
labels = df['disease'].unique()
labels = labels.tolist()

df.to_csv(labels_path, index=False)

Deleted C:/Users/x4287225/Desktop/confounder-free-cnn/OCTDL/OCTDL images/AMD\amd_1047099_1.jpg
Deleted C:/Users/x4287225/Desktop/confounder-free-cnn/OCTDL/OCTDL images/AMD\amd_1047099_2.jpg
Deleted C:/Users/x4287225/Desktop/confounder-free-cnn/OCTDL/OCTDL images/AMD\amd_1047099_3.jpg
Deleted C:/Users/x4287225/Desktop/confounder-free-cnn/OCTDL/OCTDL images/AMD\amd_1047099_4.jpg
Deleted C:/Users/x4287225/Desktop/confounder-free-cnn/OCTDL/OCTDL images/AMD\amd_1084498_1.jpg
Deleted C:/Users/x4287225/Desktop/confounder-free-cnn/OCTDL/OCTDL images/AMD\amd_1084498_2.jpg
Deleted C:/Users/x4287225/Desktop/confounder-free-cnn/OCTDL/OCTDL images/AMD\amd_1084498_3.jpg
Deleted C:/Users/x4287225/Desktop/confounder-free-cnn/OCTDL/OCTDL images/AMD\amd_1096590_1.jpg
Deleted C:/Users/x4287225/Desktop/confounder-free-cnn/OCTDL/OCTDL images/AMD\amd_1096590_2.jpg
Deleted C:/Users/x4287225/Desktop/confounder-free-cnn/OCTDL/OCTDL images/AMD\amd_1096590_3.jpg
Deleted C:/Users/x4287225/Desktop/confounder-free-

In [7]:
main()

  0%|          | 0/3 [00:00<?, ?it/s]

AMD 195 49 104


 33%|███▎      | 1/3 [00:03<00:07,  3.78s/it]

ERM 12 2 8


 67%|██████▋   | 2/3 [00:04<00:01,  1.70s/it]

NO 26 8 14


100%|██████████| 3/3 [00:04<00:00,  1.53s/it]


# Config file Prep

In [8]:

# Load the YAML file
with open(cg, 'r') as file:
    config = yaml.safe_load(file)

config['base']['data_path'] = output_folder
config['base']['save_path'] = dir + '/run' 
config['solver']['learning_rate'] = 0.0003
config['data']['num_classes'] = 3

#config['base']['device'] = 'cuda'

# Save the modified YAML back to the file
with open(cg, 'w') as file:
    yaml.safe_dump(config, file, default_flow_style=False)

# Model execution

In [9]:
def main_notebook(cfg_path, cfg_name):
    cwd = os.getcwd()
    
    # Convert the absolute config path to a relative path
    relative_cfg_path = os.path.relpath(cfg_path, cwd)
    with initialize(config_path=relative_cfg_path):
        cfg = compose(config_name=cfg_name)
        worker(cfg)

In [10]:

def worker(cfg):
    if cfg.base.random_seed != -1:
        seed = cfg.base.random_seed
        set_random_seed(seed, cfg.base.cudnn_deterministic)

    log_path = os.path.join(cfg.base.save_path, 'log')
    logger = SummaryWriter(log_path)

    # train
    model = generate_model(cfg)
    train_dataset, test_dataset, val_dataset = generate_dataset(cfg)
    estimator = Estimator(cfg.train.metrics, cfg.data.num_classes, cfg.train.criterion)
    train(
        cfg=cfg,
        model=model,
        train_dataset=train_dataset,
        val_dataset=val_dataset,
        estimator=estimator,
        logger=logger
    )
    # test
    print('Performance of the best validation model:')
    checkpoint = os.path.join(cfg.base.save_path, 'best_validation_weights.pt')
    cfg.train.checkpoint = checkpoint
    model = generate_model(cfg)
    evaluate(cfg, model, test_dataset, estimator)

    print('Performance of the final model:')
    checkpoint = os.path.join(cfg.base.save_path, 'final_weights.pt')
    cfg.train.checkpoint = checkpoint
    model = generate_model(cfg)
    evaluate(cfg, model, test_dataset, estimator)

In [11]:
def set_random_seed(seed, deterministic=False):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = deterministic

In [12]:
config_path =  dir + '/configs'
config_name = "OCTDL"
main_notebook(config_path, config_name)

C:\Users\x4287225\AppData\Local\Temp\ipykernel_24448\2007835564.py:6: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  with initialize(config_path=relative_cfg_path):


Calculating mean and std of training set for data normalization.


100%|██████████| 4/4 [00:04<00:00,  1.08s/it]


mean: [0.08435621857643127, 0.08435624092817307, 0.08435503393411636]
std: [0.15632949769496918, 0.15632951259613037, 0.15632779896259308]
Dataset Loaded.
Categories:	3
Training:	233
Validation:	59
Test:		126


  0%|          | 0/3 [00:00<?, ?it/s]c:\Users\x4287225\Desktop\confounder-free-cnn\env\Lib\site-packages\torcheval\metrics\functional\classification\accuracy.py:275: UserWarning: The reduce argument of torch.scatter with Tensor src is deprecated and will be removed in a future PyTorch release. Use torch.scatter_reduce instead for more reduction options. (Triggered internally at ..\aten\src\ATen\native\TensorAdvancedIndexing.cpp:234.)
  num_correct = mask.new_zeros(num_classes).scatter_(0, target, mask, reduce="add")
epoch: [1 / 100], loss: 1.352022: 100%|██████████| 3/3 [00:39<00:00, 13.03s/it]


...

Training metrics: acc: 0.969, f1: 0.9686, auc: 0.9972, precision: 0.969, recall: 0.969
Validation metrics:
acc: 0.833333
f1: 0.869281
auc: 1.0
precision: 0.962963
recall: 0.833333


epoch: [99 / 100], loss: 0.069763: 100%|██████████| 3/3 [00:36<00:00, 12.04s/it]


Training metrics: acc: 0.9794, f1: 0.9791, auc: 0.9984, precision: 0.9789, recall: 0.9794
Validation metrics:
acc: 0.833333
f1: 0.869281
auc: 1.0
precision: 0.962963
recall: 0.833333


epoch: [100 / 100], loss: 0.043980: 100%|██████████| 3/3 [00:35<00:00, 11.99s/it]


Training metrics: acc: 0.9899, f1: 0.9892, auc: 0.9996, precision: 0.9887, recall: 0.9899
Validation metrics:
acc: 0.833333
f1: 0.869281
auc: 1.0
precision: 0.962963
recall: 0.833333
Performance of the best validation model:
Load weights form C:/Users/x4287225/Desktop/confounder-free-cnn/OCTDL/run\best_validation_weights.pt
Running on Test set...
================Finished================
acc: 0.987179
f1: 0.951797
auc: 0.995567
precision: 0.925926
recall: 0.987179
Confusion Matrix:
[[100   0   4]
 [  0   8   0]
 [  0   0  14]]
Performance of the final model:
Load weights form C:/Users/x4287225/Desktop/confounder-free-cnn/OCTDL/run\final_weights.pt
Running on Test set...
================Finished================
acc: 0.987179
f1: 0.951797
auc: 0.995846
precision: 0.925926
recall: 0.987179
Confusion Matrix:
[[100   0   4]
 [  0   8   0]
 [  0   0  14]]
